In [ ]:
import os

from google.oauth2 import service_account
import pandas as pd
import pandas_gbq



path_to_key = os.environ["GOOGLE_APPLICATION_CREDENTIALS"]
credentials = service_account.Credentials.from_service_account_file(
    path_to_key,
)
project_id = "de-zoomcamp-451821"
dataset = "trips_data_all"

# Test connection
sql = "SELECT COUNT(*) FROM `trips_data_all.green_tripdata`;"
df = pandas_gbq.read_gbq(sql, project_id=project_id, credentials=credentials)
df

In [ ]:
url = "https://github.com/DataTalksClub/nyc-tlc-data/releases/download"
data_path = "../data/ny_taxi_data"

# taxi_type = "green"
taxi_type = "yellow"
# taxi_type = "fhv"
years = [
    "2019",
    "2020",
    ]
months = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]

for year in years:
    for month in months:
        filename = f"{taxi_type}_tripdata_{year}-{month}.csv"
        filename_gz = filename + ".gz"
        os.system(
            f"wget {url}/{taxi_type}/{filename_gz} -O {data_path}/{filename_gz}"
        )
        os.system(f"gunzip {data_path}/{filename_gz}")
        df = pd.read_csv(f"{data_path}/{filename}")
        print(f"Uploading {filename} data to BigQuery.")
        pandas_gbq.to_gbq(df, f"{dataset}.{taxi_type}_tripdata", project_id=project_id, if_exists="append")
        os.system(f"rm {data_path}/{filename}")
